In [1]:
import numpy as np
import  os 
import cv2
import matplotlib.pyplot as plt

from mtcnn import MTCNN

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


Using TensorFlow backend.


In [3]:
face_detector = MTCNN()
mask_detector = load_model('saved_model/my_model_v3.h5')

In [41]:
def detect_mask_on_video(frame, face_detector, mask_detector):
#     frame = cv2.resize(frame, (160,160))
    # find the faces present on the frame
    (f_h,f_w) = frame.shape[:2]
    detection_results = face_detector.detect_faces(frame)
    
    # Store the found faces, their coordinates and mask prediction values
    # In order to pass them to the video stream
    # So we can draw the bounding boxes and show the prediction
    
    faces = []
    bounding_boxes = []
    mask_predictions = []
    
    for i in range(len(detection_results)):
        # Get the confidence of the face prediction
        confidence  = detection_results[i]['confidence']
        
        # Check if the model is more than 50% confident about finding a face
        if confidence > 0.5:
            
            # Get the coordinates of the bounding box(location of the face)
            x,y,w,h = detection_results[i]['box']
            
#             (x, y) = (max(0, x), max(0, y))
#             (w, h) = (min(f_w - 1, w), min(f_h - 1, h))
            
            # Append the bounding box locations to its storage
            bounding_boxes.append((x,y,w,h)) 
            
            # Slice the face from the image and preprocess it
            # Resize it to 160x160 so it can match the models shape
            # pass it to the mask detection model 
            face = frame[y:y+h, x:x+w]
#             face = frame[y:h, x:w]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (160,160))
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            
            faces.append(face)
            
    if len(faces) > 0:
        # put the extracted face to mask detection model 
        # to check for a mask
        mask_predictions = mask_detector.predict(faces)
            
    return (bounding_boxes, mask_predictions)

In [44]:
# Use opencv to capture frames from video
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
#     frame = cv2.resize(frame, (400,400))
    (bounding_boxes, mask_predictions) = detect_mask_on_video(frame, face_detector, mask_detector)
    print(mask_predictions)
    
    for (b_box, m_pred) in zip(bounding_boxes, mask_predictions):
        print(m_pred)
        (x,y,w,h)  = b_box
        (no_mask, mask) = m_pred
        
        label = ''
        color = (0,0,0)
        if mask > no_mask:
            label = 'MASK'
            color = (0,255,0)
        else:
            label = 'NO MASK'
            color = (0,0,255)

        cv2.putText(frame, label, (x, y-10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x,y),(x+w, y+h), color, 2)
        
    
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

[]
[]
[[0.9689567  0.03104332]]
[0.9689567  0.03104332]
[]
[]
[]
[[0.9176242 0.0823759]]
[0.9176242 0.0823759]
[]
[]
[]
[[0.96270436 0.03729566]]
[0.96270436 0.03729566]
[]
[[0.9053029  0.09469712]]
[0.9053029  0.09469712]
[[0.8115232  0.18847685]]
[0.8115232  0.18847685]
[[0.8252958  0.17470421]]
[0.8252958  0.17470421]
[[0.89927673 0.10072324]]
[0.89927673 0.10072324]
[[0.8706758  0.12932415]]
[0.8706758  0.12932415]
[[0.7878864  0.21211362]]
[0.7878864  0.21211362]
[[0.89132404 0.10867596]]
[0.89132404 0.10867596]
[]
[[0.7618588  0.23814118]]
[0.7618588  0.23814118]
[[0.8780285  0.12197147]]
[0.8780285  0.12197147]
[[0.72043794 0.279562  ]]
[0.72043794 0.279562  ]
[[0.66904384 0.3309562 ]]
[0.66904384 0.3309562 ]
[[0.6876014  0.31239858]]
[0.6876014  0.31239858]
[[0.5766426  0.42335743]]
[0.5766426  0.42335743]
[[0.5522104  0.44778964]]
[0.5522104  0.44778964]
[[0.5141591  0.48584095]]
[0.5141591  0.48584095]
[[0.80779177 0.19220829]]
[0.80779177 0.19220829]
[[0.6188311 0.3811689]]


[[0.73553413 0.2644659 ]]
[0.73553413 0.2644659 ]
[[0.36131164 0.6386883 ]]
[0.36131164 0.6386883 ]
[[0.41902152 0.5809785 ]]
[0.41902152 0.5809785 ]
[[0.4089387  0.59106123]]
[0.4089387  0.59106123]
[[0.66545266 0.3345473 ]]
[0.66545266 0.3345473 ]


In [43]:
## If video capture somehow gets stuck, run this to end it 
cap.release()
cv2.destroyAllWindows()